## Problem 1 — Binary Words and Operations

This problem asks us to implement several low-level bitwise functions used in SHA-256.  
They are defined in the **Secure Hash Standard (FIPS 180-4)**.  
All operations must be performed on **32-bit unsigned integers**, so I use `numpy.uint32`.

The functions implemented are:

- Parity(x, y, z)
- Ch(x, y, z)
- Maj(x, y, z)
- Σ0(x), Σ1(x)
- σ0(x), σ1(x)

Each function is introduced in its own section below.


In [50]:
import numpy as np  # For 32-bit unsigned integers (np.uint32)


def rotr(x, n):
    """
    Rotate a 32-bit value x to the right by n bits.
    Used in almost all SHA-256 logical functions.
    """
    x = np.uint32(x)
    n = n % 32
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x, n):
    """
    Logical right shift of x by n bits.
    Unlike rotation, bits shifted off the right are discarded.
    """
    x = np.uint32(x)
    return np.uint32(x >> n)


## Problem 1A: Parity(x, y, z)

**Goal:** Write a function that returns the XOR of three 32-bit numbers.

From the standard (FIPS 180-4):  
**Parity(x, y, z) = x ⊕ y ⊕ z**

This function is very simple — XOR all three values.  
We ensure everything is cast to `np.uint32` so the result stays 32-bit.


In [51]:
def Parity(x, y, z):

    """Return Parity(x, y, z) = x XOR y XOR z operating on 32-bit words.



    - Inputs may be scalars or numpy arrays; they are cast to np.uint32.

    - The result preserves shape and dtype (np.uint32).

    """

    return u32(u32(x) ^ u32(y) ^ u32(z))

def Parity(x, y, z):
    """
    XOR all three 32-bit words together.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32(x ^ y ^ z)


## Problem 1B: Ch(x, y, z)

**Goal:** Implement the SHA-256 “choose” function.

From the standard:  
**Ch(x, y, z) = (x AND y) XOR (~x AND z)**

Interpretation:  
- If a bit of **x** is 1 → choose bit from **y**  
- If a bit of **x** is 0 → choose bit from **z**  


In [52]:
def Ch(x, y, z):
    """
    SHA-256 choice function:
    For each bit, choose from y if x's bit is 1, else from z.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (~x & z))


## Problem 1C: Maj(x, y, z)

**Goal:** Implement the SHA-256 majority function.

From the standard:  
**Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)**

Interpretation:  
Bit becomes **1** if *at least two* of x, y, z have a 1 in that position.


In [53]:
def Maj(x, y, z):
    """
    SHA-256 majority function.
    Bit is 1 if two or more of (x, y, z) have bit 1.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


## Problem 1D: Σ0(x) and Σ1(x)

**Goal:** Implement the two uppercase sigma functions from the standard.  

These use *rotations*, not shifts.

Definitions (FIPS 180-4):

- **Σ0(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)**
- **Σ1(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)**

These are used in the SHA-256 compression function.


In [54]:
def Sigma0(x):
    """
    Σ0(x) = ROTR^2(x) ^ ROTR^13(x) ^ ROTR^22(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


def Sigma1(x):
    """
    Σ1(x) = ROTR^6(x) ^ ROTR^11(x) ^ ROTR^25(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


## Problem 1E: σ0(x) and σ1(x)

**Goal:** Implement the lowercase sigma functions used in the message schedule.  

These mix **rotation** and **logical shift**.

Definitions (FIPS 180-4):

- **σ0(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)**  
- **σ1(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)**  

These are used when generating the message schedule array `W[t]`.

In [55]:
def sigma0(x):
    """
    σ0(x) = ROTR^7(x) ^ ROTR^18(x) ^ SHR^3(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))


def sigma1(x):
    """
    σ1(x) = ROTR^17(x) ^ ROTR^19(x) ^ SHR^10(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))


## Problem 1F: Testing all functions

To verify correctness, I test the functions using small example numbers.  
I print the results in hexadecimal (easier to compare with the standard).

In [ ]:
a = np.uint32(0x0F0F0F0F)
b = np.uint32(0x33333333)
c = np.uint32(0xAAAAAAAA)

print("Parity =", hex(a ^ b ^ c))
print("Ch     =", hex(Ch(a, b, c)))
print("Maj    =", hex(Maj(a, b, c)))

x = np.uint32(0x12345678)

print("\nSigma0 =", hex(Sigma0(x)))
print("Sigma1 =", hex(Sigma1(x)))
print("sigma0 =", hex(sigma0(x)))
print("sigma1 =", hex(sigma1(x)))


Parity = 0x96969696
Ch     = 0xa3a3a3a3
Maj    = 0x2b2b2b2b

Sigma0 = 0x66146474
Sigma1 = 0x3561abda
sigma0 = 0xe7fce6ee
sigma1 = 0xa1f78649


In [57]:
## Problem 2: Fractional Parts of Cube Roots
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [58]:
## Problem 3: Padding
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [59]:
## Problem 4: Hashes
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [60]:
## Problem 5: Passwords 
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."